<a href="https://colab.research.google.com/github/Prachii26/Autogluon/blob/main/colabs/01_tabular_quick_start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGluon Tabular - Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/tabular-quick-start.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/tabular-quick-start.ipynb)

In this tutorial, we will see how to use AutoGluon's `TabularPredictor` to predict the values of a target column based on the other columns in a tabular dataset.

Begin by making sure AutoGluon is installed, and then import AutoGluon's `TabularDataset` and `TabularPredictor`. We will use the former to load data and the latter to train models and make predictions.

In [3]:
# Clean, Colab-friendly setup for AutoGluon Tabular
!pip -q install -U pip setuptools wheel

# Optional: remove preinstalled torch/torchaudio/torchvision if they cause conflicts
!pip -q uninstall -y torchaudio torchvision torchtext > /dev/null 2>&1 || true

# Install only the Tabular component (lighter, avoids multimedia deps)
!pip -q install -U autogluon.tabular

# (Colab may prompt to restart; if it does, click "Restart runtime" and run this cell again)
import importlib.metadata as im
print("AutoGluon Tabular version:", im.version('autogluon.tabular'))


AutoGluon Tabular version: 1.4.0


In [4]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

  Using cached torchvision-0.22.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
Using cached torchvision-0.22.1-cp312-cp312-manylinux_2_28_x86_64.whl (7.5 MB)


In [5]:
from autogluon.tabular import TabularDataset, TabularPredictor

## Example Data

For this tutorial we will use a dataset from the cover story of [Nature issue 7887](https://www.nature.com/nature/volumes/600/issues/7887): [AI-guided intuition for math theorems](https://www.nature.com/articles/s41586-021-04086-x.pdf). The goal is to predict a knot's signature based on its properties. We sampled 10K training and 5K test examples from the [original data](https://github.com/deepmind/mathematics_conjectures/blob/main/knot_theory.ipynb). The sampled dataset make this tutorial run quickly, but AutoGluon can handle the full dataset if desired.

We load this dataset directly from a URL. AutoGluon's `TabularDataset` is a subclass of pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), so any `DataFrame` methods can be used on `TabularDataset` as well.

In [6]:
data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'
train_data = TabularDataset(f'{data_url}train.csv')
train_data.head()

,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


Our targets are stored in the "signature" column, which has 18 unique integers. Even though pandas didn't correctly recognize this data type as categorical, AutoGluon will fix this issue.


In [7]:
label = 'signature'
train_data[label].describe()

,signature
count,10000.000000
mean,-0.022000
std,3.025166
min,-12.000000
25%,-2.000000
50%,0.000000
75%,2.000000
max,12.000000


## Training

We now construct a `TabularPredictor` by specifying the label column name and then train on the dataset with `TabularPredictor.fit()`. We don't need to specify any other parameters. AutoGluon will recognize this is a multi-class classification task, perform automatic feature engineering, train multiple models, and then ensemble the models to create the final predictor.

In [8]:
predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20251010_221331"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Sep  6 09:54:41 UTC 2025
CPU Count:          2
Memory Avail:       11.59 GB / 12.67 GB (91.5%)
Disk Space Avail:   62.10 GB / 107.72 GB (57.6%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU me

Model fitting should take a few minutes or less depending on your CPU. You can make training faster by specifying the `time_limit` argument. For example, `fit(..., time_limit=60)` will stop training after 60 seconds. Higher time limits will generally result in better prediction performance, and excessively low time limits will prevent AutoGluon from training and ensembling a reasonable set of models.



## Prediction

Once we have a predictor that is fit on the training dataset, we can load a separate set of data to use for prediction and evaulation.

In [9]:
test_data = TabularDataset(f'{data_url}test.csv')

y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

Loaded data from: https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/test.csv | Columns = 19 / 19 | Rows = 5000 -> 5000


,signature
0,-4
1,0
2,0
3,4
4,2


## Evaluation

We can evaluate the predictor on the test dataset using the `evaluate()` function, which measures how well our predictor performs on data that was not used for fitting the models.

In [10]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.949,
 'balanced_accuracy': np.float64(0.7549527462860195),
 'mcc': np.float64(0.9375019210477166)}

AutoGluon's `TabularPredictor` also provides the `leaderboard()` function, which allows us to evaluate the performance of each individual trained model on the test data.

In [11]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.9490,0.961962,accuracy,1.983575,0.572075,106.806958,0.011906,0.001053,0.124852,2,True,12
1,LightGBM,0.9456,0.955956,accuracy,0.744830,0.137562,8.173975,0.744830,0.137562,8.173975,1,True,3
2,XGBoost,0.9448,0.956957,accuracy,1.809433,0.538760,14.287990,1.809433,0.538760,14.287990,1,True,9
3,LightGBMLarge,0.9444,0.949950,accuracy,2.297429,0.428201,15.134421,2.297429,0.428201,15.134421,1,True,11
4,CatBoost,0.9432,0.955956,accuracy,0.057359,0.012026,70.522185,0.057359,0.012026,70.522185,1,True,6
5,RandomForestEntr,0.9384,0.949950,accuracy,0.277375,0.111638,8.879235,0.277375,0.111638,8.879235,1,True,5
6,ExtraTreesGini,0.9360,0.946947,accuracy,0.397491,0.121512,2.777191,0.397491,0.121512,2.777191,1,True,7
7,ExtraTreesEntr,0.9358,0.942943,accuracy,0.456727,0.120147,2.790107,0.456727,0.120147,2.790107,1,True,8
8,RandomForestGini,0.9352,0.944945,accuracy,0.275528,0.142115,7.566166,0.275528,0.142115,7.566166,1,True,4
9,NeuralNetFastAI,0.9342,0.939940,accuracy,0.104877,0.020236,21.871931,0.104877,0.020236,21.871931,1,True,1


In [15]:
# Robust "sanity cell" to show top models and the winner
lb = predictor.leaderboard(silent=True)
display(lb.head(10))

# Pick the right score column (val/test) and find the best row
if 'score_val' in lb.columns:
    metric_col = 'score_val'
elif 'score_test' in lb.columns:
    metric_col = 'score_test'
else:
    metric_col = [c for c in lb.columns if c.startswith('score_')][0]

best_row = lb.sort_values(metric_col, ascending=False).iloc[0]
best_model = best_row['model']
best_score = best_row[metric_col]

print(f"Best model: {best_model}")
print(f"Best {metric_col}: {best_score}")


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.961962,accuracy,0.572075,106.806958,0.001053,0.124852,2,True,12
1,XGBoost,0.956957,accuracy,0.538760,14.287990,0.538760,14.287990,1,True,9
2,CatBoost,0.955956,accuracy,0.012026,70.522185,0.012026,70.522185,1,True,6
3,LightGBM,0.955956,accuracy,0.137562,8.173975,0.137562,8.173975,1,True,3
4,RandomForestEntr,0.949950,accuracy,0.111638,8.879235,0.111638,8.879235,1,True,5
5,LightGBMLarge,0.949950,accuracy,0.428201,15.134421,0.428201,15.134421,1,True,11
6,NeuralNetTorch,0.948949,accuracy,0.013250,138.411794,0.013250,138.411794,1,True,10
7,ExtraTreesGini,0.946947,accuracy,0.121512,2.777191,0.121512,2.777191,1,True,7
8,LightGBMXT,0.945946,accuracy,0.259686,13.544421,0.259686,13.544421,1,True,2
9,RandomForestGini,0.944945,accuracy,0.142115,7.566166,0.142115,7.566166,1,True,4


Best model: WeightedEnsemble_L2
Best score_val: 0.9619619619619619


## Conclusion

In this quickstart tutorial we saw AutoGluon's basic fit and predict functionality using `TabularDataset` and `TabularPredictor`. AutoGluon simplifies the model training process by not requiring feature engineering or model hyperparameter tuning. Check out the in-depth tutorials to learn more about AutoGluon's other features like customizing the training and prediction steps or extending AutoGluon with custom feature generators, models, or metrics.

In [16]:
# Version + environment stamp at the very end of the notebook
import importlib.metadata as im, sys, platform
print("AutoGluon Tabular:", im.version('autogluon.tabular'))
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())


AutoGluon Tabular: 1.4.0
Python: 3.12.11
Platform: Linux-6.6.97+-x86_64-with-glibc2.35
